In [152]:
from ews.models import BathingSpot
from django_pandas.io import read_frame
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from shapely.geometry import shape, Point
import plotly.express as px
import statsmodels


In [153]:
model = PredictionModel.objects.get(name = "ML model1")

DoesNotExist: PredictionModel matching query does not exist.

In [ ]:
PredictionModel.objects.all()

In [ ]:
areas = read_frame(model.area.all())

In [ ]:
areavars = []
for index1, row1 in areas.iterrows():
    df = read_frame(Site.objects.filter(feature_type = FeatureType.objects.get(name=row1["feature_type"])))
    polygon = shape(row1["geom"])
    select = []
    
    for index2, row2 in df.iterrows():
        select.append(polygon.contains(shape(row2['geom'])))
    data = read_frame(FeatureData.objects.filter(site__in = df[select]['id']), index_col = "date")
    data["area"] = row1["name"]
    data["feature_type"] = row1["feature_type"]
    areavars.append(data)

In [ ]:
lagvars = []

for i in range(len(areavars)):
    ft = areavars[i].area.unique()
    d = areavars[i].pivot(columns = 'site', values = 'value')
    if len(d.columns) > 1:
        d = pd.DataFrame(d.mean(axis = 1, skipna = True))
    for j in [1, 2, 3, 4, 5]:
        df = pd.DataFrame()
        df[ft + '_shift_'+ str(j)] = d.rolling(window=j).mean().shift(1)
        lagvars.append(df)
        
res = pd.concat(lagvars, axis = 1)

In [ ]:
res.filter(regex='_shift_1').plot()

In [ ]:
res = res[res.index.month.isin([ 6, 7, 8, 9])].reset_index()

In [ ]:
FIB = read_frame(FeatureData.objects.filter(site = model.site.all()[0]))

In [ ]:

d = FIB.merge(res, on= "date")

In [ ]:
D = d.dropna()

In [ ]:
y = np.log10(D["value"])

In [ ]:
X = D.drop(["date", "value", "id", "site"], axis = 1)

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, y)
y_pred= rf.predict(X)
MSE(y, y_pred)

In [ ]:
df = pd.DataFrame({'meas': y, 'pred':y_pred})

In [ ]:
fig = px.scatter(df, x = "meas", y = "pred", trendline="ols")

fig.update_layout(
    font_family="Helvetica Neue, Helvetica, Arial, sans-serif",
    font_color="black",
    title = {'text':'Model fit of Random Forest model'},
    xaxis_title = "measured data (sample)",
    yaxis_title = "fitted values (in sample fit)",
    #markercolor = "#212c52"
  
)
fig.update_traces(marker_color='#75c3ff', marker_line_color='#212c52',
                  marker_line_width=1.5, opacity=1)

fig.show()

In [ ]:
results = px.get_trendline_results(fig)
print(results)

#results.query("sex == 'Male' and smoker == 'Yes'").px_fit_results.iloc[0].summary()

In [ ]:
results.px_fit_results[0].summary()

In [ ]:
importances = pd.Series(data=rf.feature_importances_,
                        index= X.columns)

# Sort importances
importances_sorted = importances.sort_values()
importances_df = importances_sorted.reset_index()
importances_df.columns = ["feature", "importance"]

In [ ]:
# Draw a horizontal barplot of importances_sorted
import plotly.express as px

fig = px.bar(importances_df, y="feature", x="importance", orientation='h'    )

fig.update_layout(
    font_family="Helvetica Neue, Helvetica, Arial, sans-serif",
    font_color="black",
    title = {'text':'Feature importance of Random Forest model'}
    #markercolor = "#212c52"
  
)
fig.update_traces(marker_color='#75c3ff', marker_line_color='#75c3ff',
                  marker_line_width=1.5, opacity=1)

fig.show()
